In [ ]:
#| default_exp supermodel

In [ ]:
#| include: false
from nbdev.showdoc import *

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import math
import warnings

import numpy as np
from mmon_gcm.buildingediting import check_number_of_models

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

From Hills et al. (2012):
> $ VT = \frac{\sqrt{(C_{apo} + q/RT)^2 + 4pQT/RT} - C_{apo} - q/RT}{2p/RT} $ (16)

where:

$VT$ is the total volume of the guard cell

$C_{apo}$ is the sum of osmolarity in the apoplast.

$ q = n - m \cdot s / r $

where n, m, s and r are empirical parameters

$R$ is the ideal gas constant

$T$ is temperature

$ p = m/r $

QT is total osmolarity of the guard cell

They also provide a relationship between volume and aperture:

$ A_s = (VT - s)/r $ (17)

As we want a way to calculate total osmolarity from aperture we can rearrange (and just say VT is V and QT is Q) to get:

$ V = A_s \cdot r + s $

and:

$ Q = \frac{RT}{4p} \cdot \left(\left(\frac{2\cdot V\cdot p}{RT} + C_{apo} + \frac{q}{RT}\right)^2 - \left(C_{apo} + \frac{q}{RT}\right)^2\right) $

In [ ]:
#| export


class SuperModel:
    """The supermodel class is a class that includes the FBA model object as well as all the parameters used to constrain it. It provides methods to constrain the FBA model"""

    def __init__(self, parameters, fba_model=None):
        """Initialise the supermodel using the FBA model and adding the parameters from parameters as attributes"""

        if fba_model != None:
            self.fba_model = fba_model.copy()
        else:
            warnings.warn("No fba model added to the Supermodel, fine if that's what you want")

        for parameter, value in parameters.items():
            self.__setattr__(parameter, value)

        self.apertures = [
            self.A_closed,
            self.A_open,
            self.A_closed,
            self.A_closed,
        ]

    def get_volumes(self, apertures="default", printouts=False, per_guard_cell=True):
        """Returns a list containing the volume of the guard cells, in dm$^3$, at each phase, calculated using the OnGuard equation"""

        self.apertures = (
            self.__getattribute__("apertures") if apertures == "default" else apertures
        )

        # get the volume of an individual guard cell using the relationship between aperture and volume from OnGuard
        volumes_individual = [aperture * self.r + self.s for aperture in self.apertures]

        # multiply volume of an individual guard cell by the number of guard cells in 1m2 of leaf
        volumes_total = [
            volume_individual * self.N_gcs for volume_individual in volumes_individual
        ]

        if per_guard_cell == True:
            if printouts == True:
                for phase, volume in enumerate(volumes_individual):
                    volume_per_cell = volume * 10 ** 12
                    print(f"Volume in phase {phase} is {volume_per_cell}pL")

            return volumes_individual  # dm3

        else:
            if printouts == True:
                for phase, volume in enumerate(volumes_total):
                    print(f"Volume in phase {phase} is {volume}" r"dm3")

            return np.array(volumes_total)  # dm3

    def get_osmolarities(
        self, apertures="default", equation="onguard", printouts=False
    ):
        """Calculates the osmolarities at each phase of the model using the equation
        specified, default is onguard. Adds these as an attribute to the SuperModel"""

        self.apertures = (
            self.__getattribute__("apertures") if apertures == "default" else apertures
        )

        q = self.n - self.m * self.s / self.r  # atm
        p = self.m / self.r  # atm/dm3

        if equation == "onguard":
            osmolarities = [
                (
                    (
                        ((2.0 * (aperture * self.r + self.s) * p) / (self.R * self.T))
                        + self.C_apo
                        + (q / (self.R * self.T))
                    ) **
                    2.0
                    - (self.C_apo + (q / (self.R * self.T))) ** 2.0
                ) / ((4.0 * p) / (self.R * self.T))
                for aperture in self.apertures
            ]
            osmolarities = [osmolarity * self.N_gcs * 10 ** 3 for osmolarity in osmolarities] # make total os and convert to mM

        elif equation == "macrobbie":
            osmolarities = [
                (
                    (
                        (
                            (2.5 * math.exp(0.16 * aperture))
                            * ((aperture * self.r + self.s) * self.N_gcs)
                        ) /
                        (0.082 * 293)
                    ) *
                    10 ** 3
                )
                for aperture in self.apertures
            ]

        elif equation == "userset":
            osmolarities = osmolarities

        else:
            raise ValueError(equation)

        if printouts == True:
            print(f"Raw osmolarities: {osmolarities}")
            osmolarites_mM = np.array(osmolarities) / np.array(
                self.get_volumes(self.apertures, per_guard_cell=False)
            )
            print(f"Osmolarities in mM: {osmolarites_mM}")
            osmolarity_change = osmolarites_mM[1] - osmolarites_mM[0]
            print(f"Change in osmolarity: {osmolarity_change}mM")

        return osmolarities  # mmoles.m-2

    def constrain_osmolarity(
        self,
        apertures="default",
        printouts=False,
        osequation="onguard",
    ):
        """This method constrains the fba_model of the supermodel using
        osmolarities calculated using `SuperModel.get_osmolarities`"""

        self.apertures = (
            self.__getattribute__("apertures") if apertures == "default" else apertures
        )

        osmolarities = self.get_osmolarities(
            self.apertures, equation=osequation, printouts=printouts
        )

        compartments_volumes_dict = {"c": 1 - self.Vac_frac, "v": self.Vac_frac}

        for compartment, fraction in compartments_volumes_dict.items():

            compartment_osmolarity = np.array(osmolarities) * fraction

            constraints = [
                self.fba_model.problem.Constraint(
                    self.fba_model.reactions.get_by_id(
                        f"pseudoOs_constraint_{compartment}_gc_" + str(phase + 1)
                    ).flux_expression -
                    osmolarity,
                    lb=0,
                    ub=0,
                )
                for phase, osmolarity in enumerate(compartment_osmolarity)
            ]

            for constraint in constraints:
                self.fba_model.add_cons_vars(constraint)

            if printouts:
                print(f"{compartment} osmolarities constrained to {compartment_osmolarity}")

        return self

    def get_prop_gc(self, printouts=False):
        """This method returns the proportion of the volume of the leaf that is guard cells"""

        V_l = self.T_l * self.A_l  # volume of leaf is area x thickness
        V_l = V_l * 10 ** 3  # (Total leaf volume) m3 -> dm3 = 10**3

        V_gc = self.V_gc_ind * self.N_gcs  # total volume of gc in leaf

        # volume of meosphyll is leaf that isn't epidermis or air
        V_me = V_l * (1 - self.L_epidermis) * (1 - self.L_air)

        v_prop_gc = V_gc / V_me  # volume of gc is negligable

        if printouts == True:
            print(f"Total leaf volume: {V_l}dm3")
            gc_volume_rounded = round(V_gc, 4)
            print(f"Guard cell volume in 1m2 leaf: {gc_volume_rounded}dm3")
            me_volume_rounded = round(V_me, 3)
            print(f"Mesophyll cell volume in 1m2 leaf: {me_volume_rounded}dm3")
            v_prop_gc_rounded = round(v_prop_gc, 4)
            print(f"Proportion of the leaf that is gc: {v_prop_gc_rounded}")

        return v_prop_gc

    def get_photons(self, PPFD, printouts=False):
        """This method constrains photon influx into the model in accordance with
        the equation outlined in the documentation"""

        P = PPFD * self.P_abs
        P = P * 10 ** -3 * 60 * 60  # umolessec-1 -> mmoleshr-1

        e = self.FqFm * self.R_ch

        v_prop_gc = self.get_prop_gc(printouts=printouts)

        P_gc = e * v_prop_gc * P

        P_me = P - P_gc

        if printouts == True:
            print(f"PPFD {PPFD}umolphotonsm-2")
            P_influx_rounded = round(P, 3)
            print(f"Photon influx into leaf: {P_influx_rounded} mmolphotonsm-2hr-1")
            e_rounded = round(e, 3)
            print(f"e = {e_rounded}")
            P_gc_rounded = round(P_gc, 3)
            print(f"Photon influx into Guard cells: {P_gc_rounded}mmolphotonsm-2hr-1")
            P_me_rounded = round(P_me, 3)
            print(
                f"Photon influx into mesophyll cells: {P_me_rounded}mmolphotonsm-2hr-1"
            )

        return P_gc, P_me

    def constrain_photons(self, PPFD, printouts=False):
        """This method constrains the fba_model of the supermodel using
        osmolarities calculated using `SuperModel.get_osmolarities`"""

        P_gc, P_me = self.get_photons(PPFD=PPFD, printouts=printouts)

        for i, p in enumerate([0, P_gc, P_gc, 0], start=1):
            self.fba_model.reactions.get_by_id("Photon_tx_gc_" + str(i)).bounds = 0, p

        for i, p in enumerate([0, P_me, P_me, 0], start=1):
            self.fba_model.reactions.get_by_id("Photon_tx_me_" + str(i)).bounds = 0, p

        return self

    def add_maintenance(self, me=True, gc=False):
        """
        This function constrains the maintenance reactions in the model
        relative to the input of photons into the model.
        """

        model = self.fba_model

        for i in range(1, check_number_of_models(model) + 1):
            
            if me == True:
                me_maintenance = model.problem.Constraint(
                    (
                        model.reactions.get_by_id("ATPase_tx_me_" + str(i)).flux_expression
                        - (model.reactions.Photon_tx_me_3.flux_expression * 0.0049 + 2.7851)
                    ),
                    lb=0,
                    ub=1000,
                )
                model.add_cons_vars(me_maintenance)
                
            if gc == True:
                gc_maintenance = model.problem.Constraint(
                    (
                        model.reactions.get_by_id("ATPase_tx_gc_" + str(i)).flux_expression
                        - (
                            (
                                model.reactions.Photon_tx_gc_3.flux_expression * 0.0049
                                + 2.7851
                            ) *
                            self.get_prop_gc(printouts=False)
                        )
                    ),
                    lb=0,
                    ub=1000,
                )
                model.add_cons_vars(gc_maintenance)

        return self

    def get_atpase_constraint_value(self, flux_fmoles):
        '''
        This method takes the flux in fmoles/h per guard cell for the H+-ATPase and returns the flux in mmoles/h/m2 that this corresponds to for the supermodel
        '''

        flux_per_gc_moles = flux_fmoles / (10**15)
        atpase_flux = flux_per_gc_moles / (10**-3) * self.N_gcs

        return atpase_flux

In [ ]:
show_doc(SuperModel)

In [ ]:
show_doc(SuperModel.get_volumes)

Here the default is for `per_guard_cell` to be True, meaning that the list that is returned is a volume per individual cell, however setting this to false will return the colume of guard cell within the 1m$^2$ of leaf being simulated.

In [ ]:
show_doc(SuperModel.get_osmolarities)

In [ ]:
show_doc(SuperModel.constrain_osmolarity)

In [ ]:
show_doc(SuperModel.get_prop_gc)

In [ ]:
show_doc(SuperModel.get_photons)

In [ ]:
show_doc(SuperModel.constrain_photons)

In [ ]:
show_doc(SuperModel.add_maintenance)